In [ ]:
"""
Este código implementa una red neuronal convolucional en tres dimensiones (3DCNN) en keras para clasificacion
binaria. Un tutorial con una red similar puede encontrarse en  https://keras.io/examples/vision/3D_image_classification/
This code implements a 3D convolutional neural network in keras (3DCNN) for binary classification.
A similar network tutorial can be found in https://keras.io/examples/vision/3D_image_classification/

"""

In [ ]:
"""
Importar dependencias
Import dependencies
"""

import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.layers import Conv3D, BatchNormalization, MaxPool3D, GlobalAveragePooling3D, Dense, Dropout
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.optimizers import schedules, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import random
import os
import time
import numpy as np
from tensorflow.keras.preprocessing import image
from scipy.special import expit

In [2]:
"""
Comprobar la lista de GPUs activadas para el entrenamiento
Check number of GPUs activated for training
"""
len(tf.config.experimental.list_physical_devices("GPU"))

1

In [ ]:
"""
Para agilizar la carga de las imagenes estas fueron convertidas previamente en arrays de numpy
(tamaño del set x alto x ancho x largo)
To make loading the images easier, these where previosly converted into numpy arrays of size 
(size of the set x hight x width x depth)
"""

In [3]:
"""
Cargar los sets de entrenamiento, validacion y test
Load training, validation and test sets
"""

x_train = np.load("x_train_aug.npy")
y_train = np.load("y_train_aug.npy")

x_val = np.load("x_val_aug.npy")
y_val = np.load("y_val_aug.npy")

x_test = np.load("x_test_aug.npy")
y_test = np.load("y_test_aug.npy")

In [ ]:
"""
Añadir un canal: las convoluciones 3D en keras necesitan un input 
de 5 dimensiones; [batch_size, alto, ancho, profundidad, 1]. Ahora 
se añade esa quinta dimension que es 1 canal
Add a channel: 3D convolutions in keras require an input of 5 dimensions;
[batch_size, hight, widht, depth, 1]. This funtion add the five dimension
which is 1 channel
"""

def training_channel(image, label):
    image = tf.expand_dims(image, axis = 3)
    return image, label

def test_channel(image, label):
    image = tf.expand_dims(image, axis = 3)
    return image, label
    

In [ ]:
"""
Definir los loaders
Define loaders
"""
# definir los loaders

train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 8
val_batch_size = 12

# añadir el canal durante el training
# add channel on the fly of training

train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(training_channel)
    .batch(batch_size)
    .prefetch(2))

# para el validation, añadir el canal
# the same for the validation set

val_dataset = (
    val_loader.shuffle(len(x_val))
    .map(test_channel)
    .batch(batch_size)
    .prefetch(2)
)

In [ ]:
"""
Definir el modelo
Define the model
"""

def get_model(width = 95, height = 117, depth = 99):
    
    
    inputs = Input((width, height, depth, 1))
    
    x = Conv3D(8, (5,5,5), padding = "same", strides = 1, activation = "relu")(inputs)
    x = BatchNormalization()(x)

    x = Conv3D(16, (3,3,3), padding = "same", strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)
    x = MaxPool3D(pool_size = (3,3,3), strides = (2,2,2))(x)

    x = Conv3D(16, (3,3,3), padding = "same", strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)

    x = Conv3D(32, (3,3,3), padding = "same", strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((3,3,3), strides = (2,2,2))(x)

    x = Conv3D(32, (3,3,3), padding = "same",strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)

    x = Conv3D(64, (3,3,3), padding = "same", strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((3,3,3), strides = (2,2,2))(x)

    x = Conv3D(64, (3,3,3), padding = "same",strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)

    x = Conv3D(128, (3,3,3), padding = "same", strides = 1, activation = "relu")(x)
    x = BatchNormalization()(x)
    #x = MaxPool3D((3,3,3), strides = (2,2,2))(x)
    
    #x = Conv3D(128, (3,3,3), padding = "same",strides = 1, activation = "relu")(x)
    #x = BatchNormalization()(x)

    #x = Conv3D(256, (3,3,3), padding = "same", strides = 1, activation = "relu")(x)
    #x = BatchNormalization()(x)
    x = GlobalAveragePooling3D()(x)
    
    x = Dense(128, activation = "relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(64)(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(1, activation = None)(x)
    
    model = Model(inputs, outputs, name = "3Dcnn")
    return model

# Algunas capas estan comentadas porque el modelo definitivo fue menor de lo esperado
# Some layers are commented becasuse the ultimate model was smaller than expected

In [8]:
"""
Obtener objeto modelo y su resumen
Obtain model object and its summary
"""

model = get_model(width = 95, height = 117, depth = 99)
model.summary()

Model: "3Dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 95, 117, 99, 1)]  0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 95, 117, 99, 8)    1008      
_________________________________________________________________
batch_normalization_8 (Batch (None, 95, 117, 99, 8)    32        
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 95, 117, 99, 16)   3472      
_________________________________________________________________
batch_normalization_9 (Batch (None, 95, 117, 99, 16)   64        
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 47, 58, 49, 16)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 47, 58, 49, 16)    6928  

In [9]:
"""
Entrenar el modelo
Train the model
"""

learning_rate = 0.0001
batch = 8
conv_layers = 8
NAME = "3DCNN{}-lr-{}-conv-{}-batch-{}".format(learning_rate, conv_layers, batch, int(time.time()))

# NAME da nombre al archivo que guardara el modelo para la posterior monitorizacion en tensorboard
# NAME gives a name to the file in which the model will be save to be monitorized with tensorboard
tb = TensorBoard(log_dir = "logs/{}".format(NAME))

# Checkpoint guarda el mejor modelo segun la metrica elegida (monitor)
# Checkpoint saves the model accordingly to the desired metric (monitor)
checkpoint = ModelCheckpoint("best_model", monitor = "loss",  verbose=1, save_best_only=True, mode='auto', period=1)

# Compile escoge una funcion de coste (BinaryCrossentropy) , el optimizador (Adam) y la metrica (BinaryAccuracy)
# Compile chooses a cost fuction (BinaryCrossentropy), the optimizer (Adam) and the metric (BinaryAccuracy)
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = Adam(learning_rate = learning_rate),
    metrics = [tf.keras.metrics.BinaryAccuracy()],
)


# Entrenar el modelo
# Train the model

epochs = 75
model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = epochs,
    shuffle = True,
    verbose = 2,
    callbacks = [tb, checkpoint]
)

Train for 239 steps, validate for 2 steps
Epoch 1/75
239/239 - 206s - loss: 0.7490 - binary_accuracy: 0.5118 - val_loss: 0.6944 - val_binary_accuracy: 0.4615
Epoch 2/75
239/239 - 189s - loss: 0.6990 - binary_accuracy: 0.5625 - val_loss: 0.9372 - val_binary_accuracy: 0.4615
Epoch 3/75
239/239 - 190s - loss: 0.6082 - binary_accuracy: 0.6531 - val_loss: 1.4788 - val_binary_accuracy: 0.5385
Epoch 4/75
239/239 - 192s - loss: 0.4835 - binary_accuracy: 0.7614 - val_loss: 0.3611 - val_binary_accuracy: 0.6923
Epoch 5/75
239/239 - 187s - loss: 0.4176 - binary_accuracy: 0.8032 - val_loss: 0.8209 - val_binary_accuracy: 0.5385
Epoch 6/75
239/239 - 186s - loss: 0.3306 - binary_accuracy: 0.8545 - val_loss: 0.5452 - val_binary_accuracy: 0.6154
Epoch 7/75
239/239 - 185s - loss: 0.2734 - binary_accuracy: 0.8838 - val_loss: 2.6316 - val_binary_accuracy: 0.6154
Epoch 8/75
239/239 - 185s - loss: 0.2189 - binary_accuracy: 0.9074 - val_loss: 3.4769 - val_binary_accuracy: 0.5385
Epoch 9/75
239/239 - 187s - lo

Epoch 71/75
239/239 - 184s - loss: 0.0096 - binary_accuracy: 0.9969 - val_loss: 0.0072 - val_binary_accuracy: 1.0000
Epoch 72/75
239/239 - 186s - loss: 0.0185 - binary_accuracy: 0.9942 - val_loss: 0.2357 - val_binary_accuracy: 0.9231
Epoch 73/75
239/239 - 184s - loss: 0.0382 - binary_accuracy: 0.9838 - val_loss: 0.1928 - val_binary_accuracy: 0.9231
Epoch 74/75
239/239 - 186s - loss: 0.0379 - binary_accuracy: 0.9859 - val_loss: 0.1814 - val_binary_accuracy: 0.8462
Epoch 75/75
239/239 - 186s - loss: 0.0438 - binary_accuracy: 0.9833 - val_loss: 0.3869 - val_binary_accuracy: 0.7692


In [ ]:
"""
Guardar el modelo
Save the model
"""

model.save("ModelName")

In [ ]:
"""
Test set
"""

# Añadir la dimension correspondiente al canal para el test set
# Add channel dimension for the test set
x_test_channel = np.expand_dims(x_test, axis=4)

# Prediccion sobre el test set
# Prediction over the test set
preds = model.predict(x_test_channel)


# El modelo esta implementado para dar como ouput el resultado sin activacion sigmoide, eso lo hace metrics.BinaryAccuracy, más informacion (en inglés) en:
# https://medium.com/deep-learning-with-keras/which-activation-loss-functions-part-a-e16f5ad6d82a
# This model is set to give the output without sigmoid activation, that is done by metrics.BinaryAccuracy, more info in:
# https://medium.com/deep-learning-with-keras/which-activation-loss-functions-part-a-e16f5ad6d82a
print(preds)

In [ ]:
# Para obtener las predicciones en unos y ceros
# To obtain predictions in ones and zeros

BinaryPreds = expit(preds)
print(BinaryPreds)